In [1]:
from __future__ import print_function

import argparse
import os
import sys
import shutil
import time
import random

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from sklearn.metrics import top_k_accuracy_score 

from utils import Bar, Logger, AverageMeter, accuracy, mkdir_p, savefig
import numpy as np

use_cuda = torch.cuda.is_available()

print(f'Cuda: {use_cuda}')

# manualSeed = random.randint(1, 10000)
manualSeed = 0
random.seed(manualSeed)
torch.manual_seed(manualSeed)
if use_cuda:
    torch.cuda.manual_seed_all(manualSeed)

best_acc = 0  # best test accuracy

if sys.platform.startswith('linux'):
    LINUX = True
else:
    LINUX = False

if LINUX:
    DATA_ROOT = r"/home/data/sets/"
    RESULTS_ROOT = r"/home/results/privacy_reg/"
    LOG_ROOT = r"/home/logs/privacy_reg/"
else:
    DATA_ROOT = r"F:\Other\data"
    RESULTS_ROOT = r"F:\Other\privacy_reg\results"
    LOG_ROOT = r"F:\logs"
# %%
NUM_WORKERS = 0  # 0 for debugging as script?

# dataset = 'cifar10'
dataset = 'cifar100'

LR = 0.05
EPOCHS = 400

train_batch = 100
test_batch = 100
state = {'lr': LR}

checkpoint_path = os.path.join(RESULTS_ROOT, f'checkpoints_{dataset}_alexnetdefense')

C:\Users\fahim\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\fahim\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\fahim\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Cuda: True


In [ ]:
'''ResNet in PyTorch.

For Pre-activation ResNet, see 'preact_resnet.py'.

Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.autograd import Variable

class Bottleneck(nn.Module):
    expansion = 4
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out





class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=100):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])

def ResNet50():
    return ResNet(Bottleneck, [3,4,6,3])

def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])

def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])


In [ ]:
"""resnext in pytorch
[1] Saining Xie, Ross Girshick, Piotr Dollár, Zhuowen Tu, Kaiming He.
    Aggregated Residual Transformations for Deep Neural Networks
    https://arxiv.org/abs/1611.05431
"""

import math
import torch
import torch.nn as nn
import torch.nn.functional as F

#only implements ResNext bottleneck c


#"""This strategy exposes a new dimension, which we call “cardinality”
#(the size of the set of transformations), as an essential factor
#in addition to the dimensions of depth and width."""
CARDINALITY = 32
DEPTH = 4
BASEWIDTH = 64

#"""The grouped convolutional layer in Fig. 3(c) performs 32 groups
#of convolutions whose input and output channels are 4-dimensional.
#The grouped convolutional layer concatenates them as the outputs
#of the layer."""

class ResNextBottleNeckC(nn.Module):

    def __init__(self, in_channels, out_channels, stride):
        super().__init__()

        C = CARDINALITY #How many groups a feature map was splitted into

        #"""We note that the input/output width of the template is fixed as
        #256-d (Fig. 3), We note that the input/output width of the template
        #is fixed as 256-d (Fig. 3), and all widths are dou- bled each time
        #when the feature map is subsampled (see Table 1)."""
        D = int(DEPTH * out_channels / BASEWIDTH) #number of channels per group
        self.split_transforms = nn.Sequential(
            nn.Conv2d(in_channels, C * D, kernel_size=1, groups=C, bias=False),
            nn.BatchNorm2d(C * D),
            nn.ReLU(inplace=True),
            nn.Conv2d(C * D, C * D, kernel_size=3, stride=stride, groups=C, padding=1, bias=False),
            nn.BatchNorm2d(C * D),
            nn.ReLU(inplace=True),
            nn.Conv2d(C * D, out_channels * 4, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels * 4),
        )

        self.shortcut = nn.Sequential()

        if stride != 1 or in_channels != out_channels * 4:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * 4, stride=stride, kernel_size=1, bias=False),
                nn.BatchNorm2d(out_channels * 4)
            )

    def forward(self, x):
        return F.relu(self.split_transforms(x) + self.shortcut(x))

class ResNext(nn.Module):

    def __init__(self, block, num_blocks, class_names=100):
        super().__init__()
        self.in_channels = 64

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

        self.conv2 = self._make_layer(block, num_blocks[0], 64, 1)
        self.conv3 = self._make_layer(block, num_blocks[1], 128, 2)
        self.conv4 = self._make_layer(block, num_blocks[2], 256, 2)
        self.conv5 = self._make_layer(block, num_blocks[3], 512, 2)
        self.avg = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(512 * 4, 100)
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.avg(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x= self.fc2(x)
        return x

    def _make_layer(self, block, num_block, out_channels, stride):
        """Building resnext block
        Args:
            block: block type(default resnext bottleneck c)
            num_block: number of blocks per layer
            out_channels: output channels per block
            stride: block stride
        Returns:
            a resnext layer
        """
        strides = [stride] + [1] * (num_block - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * 4

        return nn.Sequential(*layers)

def resnext50():
    """ return a resnext50(c32x4d) network
    """
    return ResNext(ResNextBottleNeckC, [3, 4, 6, 3])

def resnext101():
    """ return a resnext101(c32x4d) network
    """
    return ResNext(ResNextBottleNeckC, [3, 4, 23, 3])

def resnext152():
    """ return a resnext101(c32x4d) network
    """
    return ResNext(ResNextBottleNeckC, [3, 4, 36, 3])


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from torch.autograd import Variable

# __all__ = ['densenet']


class Bottleneck(nn.Module):
    def __init__(self, inplanes, expansion=4, growthRate=12, dropRate=0):
        super(Bottleneck, self).__init__()
        planes = expansion * growthRate
        self.bn1 = nn.BatchNorm2d(inplanes)
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, growthRate, kernel_size=3, 
                               padding=1, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.dropRate = dropRate

    def forward(self, x):
        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv2(out)
        if self.dropRate > 0:
            out = F.dropout(out, p=self.dropRate, training=self.training)

        out = torch.cat((x, out), 1)

        return out


class BasicBlock(nn.Module):
    def __init__(self, inplanes, expansion=1, growthRate=12, dropRate=0):
        super(BasicBlock, self).__init__()
        planes = expansion * growthRate
        self.bn1 = nn.BatchNorm2d(inplanes)
        self.conv1 = nn.Conv2d(inplanes, growthRate, kernel_size=3, 
                               padding=1, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.dropRate = dropRate

    def forward(self, x):
        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)
        if self.dropRate > 0:
            out = F.dropout(out, p=self.dropRate, training=self.training)

        out = torch.cat((x, out), 1)

        return out


class Transition(nn.Module):
    def __init__(self, inplanes, outplanes):
        super(Transition, self).__init__()
        self.bn1 = nn.BatchNorm2d(inplanes)
        self.conv1 = nn.Conv2d(inplanes, outplanes, kernel_size=1,
                               bias=False)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)
        out = F.avg_pool2d(out, 2)
        return out


class DenseNet(nn.Module):
#it was 12 (growth rate) now 19
    def __init__(self, depth=22, block=Bottleneck, 
        dropRate=0, num_classes=100, growthRate=19, compressionRate=2):
        super(DenseNet, self).__init__()

        assert (depth - 4) % 3 == 0, 'depth should be 3n+4'
        n = (depth - 4) / 3 if block == BasicBlock else (depth - 4) // 6

        self.growthRate = growthRate
        self.dropRate = dropRate

        # self.inplanes is a global variable used across multiple
        # helper functions
        self.inplanes = growthRate * 2 
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=3, padding=1,
                               bias=False)
        self.dense1 = self._make_denseblock(block, n)
        self.trans1 = self._make_transition(compressionRate)
        self.dense2 = self._make_denseblock(block, n)
        self.trans2 = self._make_transition(compressionRate)
        self.dense3 = self._make_denseblock(block, n)
        self.bn = nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.avgpool = nn.AvgPool2d(8)
        self.fc = nn.Linear(self.inplanes, num_classes)

        # Weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_denseblock(self, block, blocks):
        layers = []
        for i in range(blocks):
            # Currently we fix the expansion ratio as the default value
            layers.append(block(self.inplanes, growthRate=self.growthRate, dropRate=self.dropRate))
            self.inplanes += self.growthRate

        return nn.Sequential(*layers)

    def _make_transition(self, compressionRate):
        inplanes = self.inplanes
        outplanes = int(math.floor(self.inplanes // compressionRate))
        self.inplanes = outplanes
        return Transition(inplanes, outplanes)


    def forward(self, x):
        x = self.conv1(x)

        x = self.trans1(self.dense1(x)) 
        x = self.trans2(self.dense2(x)) 
        x = self.dense3(x)
        x = self.bn(x)
        x = self.relu(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [ ]:
# %% AlexNet Module


class AlexNet(nn.Module):

    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 192, kernel_size=(5, 5), padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(192, 384, kernel_size=(3, 3), padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=(3, 3), padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


def alexnet(**kwargs):
    r"""AlexNet model architecture from the
    `"One weird trick..." <https://arxiv.org/abs/1404.5997>`_ paper.
    """
    model = AlexNet(**kwargs)
    return model




In [ ]:
# %% Inference Attack HZ Class


class InferenceAttack_HZ(nn.Module):
    def __init__(self, num_classes):
        self.num_classes = num_classes

        super(InferenceAttack_HZ, self).__init__()

        self.features = nn.Sequential(
            nn.Linear(self.num_classes, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 64),
            nn.ReLU(),
        )

        self.labels = nn.Sequential(
            nn.Linear(num_classes, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
        )

        self.combine = nn.Sequential(
            nn.Linear(64 * 2, 256),

            nn.ReLU(),
            nn.Linear(256, 128),

            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
        )

        for key in self.state_dict():
            print(f'\t {key}')
            if key.split('.')[-1] == 'weight':
                nn.init.normal_(self.state_dict()[key], std=0.01)

            elif key.split('.')[-1] == 'bias':
                self.state_dict()[key][...] = 0

        self.output = nn.Sigmoid()

    def forward(self, x, labels):

        out_x = self.features(x)
        out_l = self.labels(labels)

        is_member = self.combine(torch.cat((out_x, out_l), 1))

        return self.output(is_member)


# %% Status Func

def report_str(batch_idx, data_time, batch_time, losses, top1, top5):
    batch = f'({batch_idx:4d})'
    time = f'Data: {data_time:.2f}s | Batch: {batch_time:.2f}s'
    loss_ac1 = f'Loss: {losses:.3f} | Top1: {top1 * 100:.2f}%'

    res = f'{batch} {time} || {loss_ac1}'

    if top5 is None:
        return res
    else:
        return res + f' | Top5: {top5 * 100:.2f}%'

    

In [ ]:
# %% train_privately
#alpha=0.9
def train_privately(trainloader, model, inference_model, criterion, optimizer, use_cuda, num_batches=10000, alpha=1.9):
    # switch to train mode
    model.train()
    inference_model.eval()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    end = time.time()

    first_id = -1
    for batch_idx, (inputs, targets) in trainloader:
        # measure data loading time
        data_time.update(time.time() - end)
        if first_id == -1:
            first_id = batch_idx

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda(non_blocking=True)
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)

        one_hot_tr = torch.from_numpy((np.zeros((outputs.size(0), num_classes)) - 1)).cuda().type(torch.float)

        target_one_hot_tr = one_hot_tr.scatter_(1, targets.type(torch.int64).view([-1, 1]).data, 1)

        infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)

        inference_output = inference_model(outputs, infer_input_one_hot)
        # print (inference_output.mean())
        loss = criterion(outputs, targets) + alpha * ((inference_output - 1.0).pow(2).mean())

        # measure accuracy and record loss
        # prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))

        prec1 = top_k_accuracy_score(y_true=targets.data.cpu(), y_score=outputs.data.cpu(),
                                     k=1, labels=range(num_classes))

        prec5 = top_k_accuracy_score(y_true=targets.data.cpu(), y_score=outputs.data.cpu(),
                                     k=5, labels=range(num_classes))

        losses.update(loss.data.item(), inputs.size(0))
        top1.update(prec1, inputs.size(0))
        top5.update(prec5, inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if batch_idx % 100 == 0:
            print(report_str(batch_idx + 1, data_time.avg, batch_time.avg, losses.avg, top1.avg, top5.avg))

        if batch_idx - first_id >= num_batches:
            break

    return losses.avg, top1.avg


In [ ]:
# %% train
def train(trainloader, model, criterion, optimizer, use_cuda):
    # switch to train mode
    model.train()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    end = time.time()

    bar = Bar('Processing', max=len(trainloader))
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda(non_blocking=True)
            inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)

        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        # prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))

        prec1 = top_k_accuracy_score(y_true=targets.data.cpu(), y_score=outputs.data.cpu(),
                                     k=1, labels=range(num_classes))

        prec5 = top_k_accuracy_score(y_true=targets.data.cpu(), y_score=outputs.data.cpu(),
                                     k=5, labels=range(num_classes))

        losses.update(loss.data.item(), inputs.size(0))
        top1.update(prec1, inputs.size(0))
        top5.update(prec5, inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if batch_idx % 100 == 0:
            print(report_str(batch_idx + 1, data_time.avg, batch_time.avg, losses.avg, top1.avg, top5.avg))

        return losses.avg, top1.avg



In [ ]:
# %% test
def test(testloader, model, criterion, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs = inputs.cuda()
                targets = targets.cuda()

            # compute output
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # measure accuracy and record loss
            # prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
            prec1 = top_k_accuracy_score(y_true=targets.data.cpu(), y_score=outputs.data.cpu(),
                                         k=1, labels=range(num_classes))

            prec5 = top_k_accuracy_score(y_true=targets.data.cpu(), y_score=outputs.data.cpu(), k=5,
                                         labels=range(num_classes))
            losses.update(loss.data.item(), inputs.size(0))
            top1.update(prec1, inputs.size(0))
            top5.update(prec5, inputs.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            # plot progress
            if batch_idx % 100 == 0:
                print(report_str(batch_idx + 1, data_time.avg, batch_time.avg, losses.avg, top1.avg, top5.avg))

    return losses.avg, top1.avg

In [ ]:
# %% privacy_train
def privacy_train(trainloader, model, inference_model, criterion, optimizer, use_cuda, num_batchs=1000):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    mtop1_a = AverageMeter()
    mtop5_a = AverageMeter()

    inference_model.train()
    model.eval()
    # switch to evaluate mode

    end = time.time()
    first_id = -1
    for batch_idx, ((tr_input, tr_target), (te_input, te_target)) in trainloader:
        # measure data loading time
        if first_id == -1:
            first_id = batch_idx

        data_time.update(time.time() - end)

        if use_cuda:
            tr_input = tr_input.cuda()
            te_input = te_input.cuda()
            tr_target = tr_target.cuda()
            te_target = te_target.cuda()

        v_tr_input = torch.autograd.Variable(tr_input)
        v_te_input = torch.autograd.Variable(te_input)
        v_tr_target = torch.autograd.Variable(tr_target)
        v_te_target = torch.autograd.Variable(te_target)

        # compute output
        model_input = torch.cat((v_tr_input, v_te_input))

        pred_outputs = model(model_input)
        #y_hat

        infer_input = torch.cat((v_tr_target, v_te_target))
        #(y_hat)

        # TODO fix
        # mtop1, mtop5 = accuracy(pred_outputs.data, infer_input.data, topk=(1, 5))
        mtop1 = top_k_accuracy_score(y_true=infer_input.data.cpu(), y_score=pred_outputs.data.cpu(),
                                     k=1, labels=range(num_classes))

        mtop5 = top_k_accuracy_score(y_true=infer_input.data.cpu(), y_score=pred_outputs.data.cpu(),
                                     k=5, labels=range(num_classes))

        mtop1_a.update(mtop1, model_input.size(0))
        mtop5_a.update(mtop5, model_input.size(0))

        one_hot_tr = torch.from_numpy((np.zeros((infer_input.size(0), num_classes)) - 1)).cuda().type(torch.float)
        target_one_hot_tr = one_hot_tr.scatter_(1, infer_input.type(torch.int64).view([-1, 1]).data, 1)

        infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)
        #ONE_hot y_hat

        attack_model_input = pred_outputs  # torch.cat((pred_outputs,infer_input_one_hot),1)
        member_output = inference_model(attack_model_input, infer_input_one_hot)
        #inf_model(y,y_hat)
        #member->?0/1

        is_member_labels = torch.from_numpy(
            np.reshape(
                np.concatenate((np.zeros(v_tr_input.size(0)), np.ones(v_te_input.size(0)))),
                [-1, 1]
            )
        ).cuda()

        v_is_member_labels = torch.autograd.Variable(is_member_labels).type(torch.float)
        #true_labels

        loss = criterion(member_output, v_is_member_labels)

        # measure accuracy and record loss
        prec1 = np.mean((member_output.data.cpu().numpy() > 0.5) == v_is_member_labels.data.cpu().numpy())
        losses.update(loss.data.item(), model_input.size(0))
        top1.update(prec1, model_input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if batch_idx - first_id > num_batchs:
            break

        # plot progress
        if batch_idx % 10 == 0:
            print(report_str(batch_idx, data_time.avg, batch_time.avg, losses.avg, top1.avg, None))

    return losses.avg, top1.avg



In [ ]:
# %% privacy_test
def privacy_test(trainloader, model, inference_model, criterion, optimizer, use_cuda, num_batchs=1000):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    mtop1_a = AverageMeter()
    mtop5_a = AverageMeter()

    inference_model.eval()
    model.eval()
    # switch to evaluate mode

    end = time.time()
    first_id = -1
    for batch_idx, ((tr_input, tr_target), (te_input, te_target)) in trainloader:
        # measure data loading time
        if first_id == -1:
            first_id = batch_idx

        data_time.update(time.time() - end)

        if use_cuda:
            tr_input = tr_input.cuda()
            te_input = te_input.cuda()
            tr_target = tr_target.cuda()
            te_target = te_target.cuda()

        v_tr_input = torch.autograd.Variable(tr_input)
        v_te_input = torch.autograd.Variable(te_input)
        v_tr_target = torch.autograd.Variable(tr_target)
        v_te_target = torch.autograd.Variable(te_target)

        # compute output
        model_input = torch.cat((v_tr_input, v_te_input))

        pred_outputs = model(model_input)

        infer_input = torch.cat((v_tr_target, v_te_target))

        # mtop1, mtop5 = accuracy(pred_outputs.data, infer_input.data, topk=(1, 5))
        mtop1 = top_k_accuracy_score(y_true=pred_outputs.data.cpu(), y_score=infer_input.data.cpu(), k=1,
                                     labels=range(num_classes))
        mtop5 = top_k_accuracy_score(y_true=pred_outputs.data.cpu(), y_score=infer_input.data.cpu(), k=5,
                                     labels=range(num_classes))

        mtop1_a.update(mtop1, model_input.size(0))
        mtop5_a.update(mtop5, model_input.size(0))

        one_hot_tr = torch.from_numpy((np.zeros((infer_input.size(0), num_classes)) - 1)).cuda().type(torch.float)
        target_one_hot_tr = one_hot_tr.scatter_(1, infer_input.type(torch.int64).view([-1, 1]).data, 1)

        infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)

        attack_model_input = pred_outputs  # torch.cat((pred_outputs,infer_input_one_hot),1)
        member_output = inference_model(attack_model_input, infer_input_one_hot)

        is_member_labels = torch.from_numpy(
            np.reshape(
                np.concatenate((np.zeros(v_tr_input.size(0)), np.ones(v_te_input.size(0)))),
                [-1, 1]
            )
        ).cuda()

        v_is_member_labels = torch.autograd.Variable(is_member_labels).type(torch.float)

        loss = criterion(member_output, v_is_member_labels)

        # measure accuracy and record loss
        prec1 = np.mean((member_output.data.cpu().numpy() > 0.5) == v_is_member_labels.data.cpu().numpy())
        losses.update(loss.data.item(), model_input.size(0))
        top1.update(prec1, model_input.size(0))

        # compute gradient and do SGD step

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if batch_idx - first_id >= num_batchs:
            break

        # plot progress
        # if batch_idx%10==0:
        #     print(report_str(batch_idx + 1, data_time.avg, batch_time.avg, losses.avg, top1.avg, None))
        #     print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
        #             batch=batch_idx ,
        #             size=len(trainloader),
        #             data=data_time.avg,
        #             bt=batch_time.avg,
        #             loss=losses.avg,
        #             top1=top1.avg,
        #             ))

    return losses.avg, top1.avg



In [ ]:
# %% checkpoint, adjust LR
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    global state
    if epoch in [20, 40]:
        state['lr'] *= 0.1
        for param_group in optimizer.param_groups:
            param_group['lr'] = state['lr']


def save_checkpoint_adversary(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_adversary_best.pth.tar'))


# %% Dataset
dataset ='cifar10'
print('==> Preparing dataset %s' % dataset)

In [ ]:
import model.resnet as resnet
import model.resnext as resnext

if dataset == 'cifar10':
    train_mean = np.array([125.307, 122.950, 113.865])
    train_std = np.array([62.993, 62.089, 66.705])
    test_mean = np.array([126.025, 123.708, 114.854])
    test_std = np.array([62.896, 61.937, 66.706])
else:
    train_ds = datasets.CIFAR100(os.path.join(DATA_ROOT, 'cifar100'), train=True, download=True)
    test_ds = datasets.CIFAR100(os.path.join(DATA_ROOT, 'cifar100'), train=False, download=True)

    _data_train = np.concatenate([np.array(train_ds[i][0]) for i in range(len(train_ds))])
    _data_test = np.concatenate([np.array(test_ds[i][0]) for i in range(len(test_ds))])

    train_mean = _data_train.mean(axis=(0, 1))
    train_std = _data_train.std(axis=(0, 1))

    test_mean = _data_test.mean(axis=(0, 1))
    test_std = _data_test.std(axis=(0, 1))
    
#     train_mean = (0.5070751592371323, 0.48654887331495095, 0.4409178433670343)
#     train_std = (0.2673342858792401, 0.2564384629170883, 0.27615047132568404)
    
#     test_mean = (0.5088964127604166, 0.48739301317401956, 0.44194221124387256)
#     test_mean = (0.2682515741720801, 0.2573637364478126, 0.2770957707973042)

    print(f'Hard code CIFAR100 train/test mean/std for next time')

print('train mean/std:', train_mean, train_std)
print('test mean/std:', test_mean, test_std)

# Normalize mean std to 0..1 from 0..255
train_mean /= 255
train_std /= 255
test_mean /= 255
test_std /= 255

transform_train = transforms.Compose([
    # transforms.RandomCrop(32, padding=4),
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])

transform_test = transforms.Compose([
    
    transforms.ToTensor(),
    transforms.Normalize(test_mean, test_std),
])

# %% Choose model params from dataset

if dataset == 'cifar10':
    dataloader = datasets.CIFAR10
    data_loader_root = os.path.join(DATA_ROOT, 'cifar10')
    num_classes = 10
    title = 'cifar-10'
else:
    dataloader = datasets.CIFAR100
    data_loader_root = os.path.join(DATA_ROOT, 'cifar100')
    num_classes = 100
    title = 'cifar-100'

# %% Models, criterions, optimizers

print("==> creating model ")
# model = AlexNet(num_classes)
#model=resnet.ResNet50()
#model=resnext.CifarResNeXt(cardinality=32,depth=4)
model=resnext50()
# model=DenseNet(growthRate=19)
model = model.cuda()

# inference_model = torch.nn.DataParallel(inferenece_model).cuda()
cudnn.benchmark = True
print('\tTotal params: %.2fM' % (sum(p.numel() for p in model.parameters()) / 1000000.0))

criterion = nn.CrossEntropyLoss()

criterion_attack = nn.MSELoss()

optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)

inference_model = InferenceAttack_HZ(num_classes).cuda()

private_train_criterion = nn.MSELoss()

optimizer_mem = optim.Adam(inference_model.parameters(), lr=0.00001)

# %% Load Dataset from file
print("==> Loading selected datasets")
batch_privacy = 100
trainset = dataloader(root=data_loader_root, train=True, download=True, transform=transform_train)
trainloader = data.DataLoader(trainset, batch_size=batch_privacy, shuffle=True, num_workers=NUM_WORKERS)

# TODO check loader for trainloader_private
trainset_private = dataloader(root=data_loader_root, train=True, download=True, transform=transform_test)
trainloader_private = data.DataLoader(trainset, batch_size=batch_privacy, shuffle=True, num_workers=NUM_WORKERS)

testset = dataloader(root=data_loader_root, train=False, download=False, transform=transform_test)
testloader = data.DataLoader(testset, batch_size=batch_privacy, shuffle=True, num_workers=NUM_WORKERS)

In [ ]:
# start_epoch = 0  # start from epoch 0 or last checkpoint epoch
if not os.path.isdir(checkpoint_path):
    mkdir_p(checkpoint_path)

is_best = False
best_acc = 0.0
start_epoch = 0

# Train and val
for epoch in range(start_epoch, EPOCHS):
    adjust_learning_rate(optimizer, epoch)

    print(f'\nEpoch: [{epoch + 1:d} | {EPOCHS:d}] LR: {state["lr"]:f}')

    train_enum = enumerate(trainloader)
    train_private_enum = enumerate(zip(trainloader_private, testloader))
    for i in range(500 // 2):

        if epoch > 3:
            privacy_loss, privacy_acc = privacy_train(
                train_private_enum, model, inference_model, criterion_attack, optimizer_mem, use_cuda, 1
            )

            train_loss, train_acc = train_privately(
                train_enum, model, inference_model, criterion, optimizer, use_cuda, 1, 1
            )

            if i % 10 == 0:
                print(f'Privacy Res: {privacy_acc * 100:.2f}% | {train_acc * 100:.2f}%')

            if (i + 1) % 50 == 0:
                train_private_enum = enumerate(zip(trainloader_private, testloader))
        else:
            train_loss, train_acc = train_privately(
                train_enum, model, inference_model, criterion, optimizer, use_cuda, 1000, 0
            )

            break

    test_loss, test_acc = test(testloader, model, criterion, use_cuda)

    print(f'Test Acc: {test_acc * 100:.2f}%')

    # save model
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epochz': epoch + 1,
        'state_dict': model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, False, checkpoint=checkpoint_path, filename='epoch%d' % epoch)

print(f'Best acc: {best_acc * 100:.2f}%')


In [ ]:
print(model)

In [ ]:
# use GPU if available
import utilse

utilse.load_checkpoint('E:/KD_MI/experiments/base_cnn/cifar10/resnext/epoch14',model)